In [ ]:
import pickle
import numpy as np
import pandas as pd
from os import listdir
from os.path import join, isfile, expandvars

# data
table = 'regressor_5sgm_exp30m_z20_xy_flip.pickle'
path = f'{expandvars("$HOME")}/E4/irf_z20/crab/'
dataset = join(path, table)

# dataset 
if '.pickle' in table:
    with open(dataset,'rb') as f: ds = pickle.load(f)
    infotable = join(path, table.replace('.pickle', '.dat'))
elif '.npy' in table:
    ds = np.load(dataset, allow_pickle=True, encoding='latin1', fix_imports=True).flat[0]
    infotable = join(path, table.replace('.npy', '.dat'))


print(f"Maps: {ds.keys()}")
print(f"DS dataset size: {len(ds['DS'])}")

In [ ]:
from astroai.tools.utils import split_regression_dataset

train_data, train_labels, test_data, test_labels = split_regression_dataset(ds, split=80, reshape=True, binning=200)

print(f"Train dataset: {len(train_data)}, {train_data.shape}")
print(f"Train labels: {len(train_labels)}, {train_labels.shape}")
print(f"\nTest dataset: {len(test_data)}, {test_data.shape}")
print(f"Test labels: {len(test_labels)}, {test_labels.shape}")

# Check 1 seed

In [ ]:
import numpy as np
from astroai.tools.utils import plot_heatmap, set_wcs

# get random seed
idx = np.random.choice(range(len(test_data)))
# find seed to get the original heatmap
seed = len(train_data) + idx + 1

# get simulation data
infodata = pd.read_csv(infotable, sep=' ', header=0).sort_values(by=['seed'])
row = infodata[infodata['seed']==seed]
row

In [ ]:
from astroai.tools.utils import *

binning = 200
pixelsize = (2 * row['fov'].values[0]) / binning
point_ref = (binning / 2) + (pixelsize / 2)
w = set_wcs(point_ra=row['point_ra'].values[0], point_dec=row['point_dec'].values[0], 
            point_ref=point_ref, pixelsize=pixelsize)

# TRUE
true_sky = SkyCoord(ra=row['source_ra'].values[0], dec=row['source_dec'].values[0], unit='deg', frame='icrs')
x, y = w.world_to_pixel(true_sky) 
true_sky = true_sky.ra.deg, true_sky.dec.deg
true_pix = x, y

# LABEL
label_pix = test_labels[idx][0] * binning, test_labels[idx][1] * binning
sky = w.pixel_to_world(label_pix[0], label_pix[1])
label_sky = sky.ra.deg, sky.dec.deg

print(f'SEED: {seed}')
print('---- PIX')
print(f"true: {true_pix}")
print(f"label: {label_pix}")
print('---- SKY')
print(f"true: {true_sky} ")
print(f"label: {label_sky}")

In [ ]:
import astropy.units as u
from matplotlib.lines import Line2D

radius_deg = 0.2
radius_pix = radius_deg/0.025
figsize = (6, 6)

# PLOT
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(121), fig.add_subplot(122, projection=w)

# PIX
ax[0].imshow(test_data[idx], vmin=0, vmax=1)
ax[0].add_patch(plt.Circle((true_pix), radius=radius_pix, edgecolor='r', facecolor='none'))
ax[0].add_patch(plt.Circle((label_pix), radius=radius_pix, edgecolor='k', facecolor='none', ls='-.'))
ax[0].set_ylabel('y [pixels]')
ax[0].set_xlabel('x [pixels]')

# SKY
ax[1].imshow(test_data[idx], vmin=0, vmax=1)
ax[1].scatter(true_sky[0], true_sky[1], transform=ax[1].get_transform('icrs'), s=300,
               edgecolor='r', facecolor='none')
ax[1].scatter(label_sky[0], label_sky[1], transform=ax[1].get_transform('icrs'), s=300,
               edgecolor='k', facecolor='none', ls='-.')
ax[1].set_ylabel('y [pixels]')
ax[1].set_xlabel('x [pixels]')

# LEGENDS
custom_lines = [Line2D([0], [0], color='r', lw=1, ls='-'),
                Line2D([0], [0], color='k', lw=1, ls='-.')]
ax[0].legend(custom_lines, ['true', 'label'])
ax[1].legend(custom_lines, ['true', 'label'])

plt.show()